# 스타벅스 이디야 데이터 분석 - SQL 사용하기

In [290]:
import mysql.connector
import pandas as pd



In [291]:
mydb = mysql.connector.connect(
host='database-1.cf7lbkcygbj1.ap-northeast-1.rds.amazonaws.com',
port=3306,
user='admin',
password='*****',
)

In [292]:
cur = mydb.cursor()

In [177]:
cur.execute('SHOW CREATE DATABASE oneday')
result=cur.fetchall()
print(result)

[('oneday', "CREATE DATABASE `oneday` /*!40100 DEFAULT CHARACTER SET utf8mb4 COLLATE utf8mb4_0900_ai_ci */ /*!80016 DEFAULT ENCRYPTION='N' */")]


In [286]:
cur.execute('SHOW GRANTS FOR "oneday"@"%"')
result=cur.fetchall()
print(result)

[('GRANT USAGE ON *.* TO `oneday`@`%`',)]


In [293]:
cur.execute('use oneday')

In [168]:
query='Desc COFFEE_BRAND'
df = pd.read_sql(query,mydb)
print(df)

  Field            Type Null  Key Default           Extra
0    id          b'int'   NO  PRI    None  auto_increment
1  name  b'varchar(10)'  YES         None                


C:\Users\Ssoony\AppData\Local\Temp\ipykernel_22076\2910473203.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query,mydb)


In [169]:
query='Desc COFFEE_STORE'
df = pd.read_sql(query,mydb)
print(df)

     Field               Type Null  Key Default           Extra
0       id             b'int'   NO  PRI    None  auto_increment
1    brand             b'int'  YES  MUL    None                
2     name     b'varchar(32)'  YES         None                
3  gu_name      b'varchar(5)'  YES         None                
4  address    b'varchar(128)'  YES         None                
5      lat  b'decimal(16,14)'  YES         None                
6      lng  b'decimal(17,14)'  YES         None                


C:\Users\Ssoony\AppData\Local\Temp\ipykernel_22076\3012202072.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query,mydb)


In [48]:
cur.execute('INSERT INTO COFFEE_BRAND(name)value("STARBUCKS")')
cur.execute('INSERT INTO COFFEE_BRAND(name)value("EDIYA")')


In [ ]:
mydb.commit()

In [138]:
query="SELECT*FROM COFFEE_BRAND"
df = pd.read_sql(query,mydb)
print(df)

   id       name
0   1  STARBUCKS
1   2      EDIYA


C:\Users\Ssoony\AppData\Local\Temp\ipykernel_22076\1419203009.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query,mydb)


In [233]:

from selenium import webdriver
from bs4 import BeautifulSoup
import time
import pandas as pd
from tqdm import tqdm_notebook

In [234]:

driver=webdriver.Chrome('../driver/chromedriver.exe')
driver.get('https://www.starbucks.co.kr/store/store_map.do?disp=locale')
time.sleep(2)

In [235]:
try:
    driver.find_element_by_css_selector('.holiday_notice_close a').click()
except Exception as errormsg:
    print(errormsg)

Message: no such element: Unable to locate element: {"method":"css selector","selector":".holiday_notice_close a"}
  (Session info: chrome=108.0.5359.125)



In [236]:
driver.find_element_by_xpath('//*[@id="container"]/div/form/fieldset/div/section/article[1]/article/article[2]/div[1]/div[2]/ul/li[1]/a').click()

In [237]:
gugun=driver.find_element_by_class_name('gugun_arae_box')
gugun_list=gugun.find_elements_by_tag_name('li')
time.sleep(5)

In [238]:
gugun_list[0].click()
time.sleep(5)

In [239]:
starbucksCode=driver.page_source
bs=BeautifulSoup(starbucksCode,'lxml')
starbucksgugun=bs.find('ul',class_='quickSearchResultBoxSidoGugun')
starbuckrawData=starbucksgugun.find_all('li')
for num1 in tqdm_notebook(starbuckrawData):
    store_name=num1.find('strong').text
    store_address=num1.find('p').text[:-9]
    store_guname=store_address.split(' ')[1]
    store_lat=num1['data-lat'].strip()
    store_lng=num1['data-long'].strip()
    cur.execute(f'INSERT INTO COFFEE_STORE(brand,name,gu_name,address,lat,lng) VALUES(1,\"{store_name}\",\"{store_guname}\",\"{store_address}\",{store_lat},{store_lng})')
    mydb.commit()
    time.sleep(1)


C:\Users\Ssoony\AppData\Local\Temp\ipykernel_22076\3617109532.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for num1 in tqdm_notebook(starbuckrawData):


  0%|          | 0/598 [00:00<?, ?it/s]

In [240]:
driver.close()

In [181]:
query='select COUNT(*) from COFFEE_STORE'
df = pd.read_sql(query,mydb)
print(df)

   COUNT(*)
0       598


C:\Users\Ssoony\AppData\Local\Temp\ipykernel_22076\3406107624.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query,mydb)


In [180]:
query='select * from COFFEE_STORE limit 10'
df = pd.read_sql(query,mydb)
print(df)

   id  brand        name gu_name                     address        lat  \
0   1      1   역삼아레나빌딩       강남구     서울특별시 강남구 언주로 425 (역삼동)  37.501087   
1   2      1    논현역사거리       강남구    서울특별시 강남구 강남대로 538 (논현동)  37.510178   
2   3      1   신사역성일빌딩       강남구    서울특별시 강남구 강남대로 584 (논현동)  37.513931   
3   4      1    국기원사거리       강남구    서울특별시 강남구 테헤란로 125 (역삼동)  37.499517   
4   5      1   대치재경빌딩R       강남구  서울특별시 강남구 남부순환로 2947 (대치동)  37.494668   
5   6      1      봉은사역       강남구    서울특별시 강남구 봉은사로 619 (삼성동)  37.515000   
6   7      1   압구정윤성빌딩       강남구     서울특별시 강남구 논현로 834 (신사동)  37.522793   
7   8      1    코엑스별마당       강남구    서울특별시 강남구 영동대로 513 (삼성동)  37.510150   
8   9      1  삼성역섬유센터R       강남구    서울특별시 강남구 테헤란로 518 (대치동)  37.507750   
9  10      1      압구정R       강남구     서울특별시 강남구 언주로 861 (신사동)  37.527367   

          lng  
0  127.043069  
1  127.022223  
2  127.020606  
3  127.031495  
4  127.062583  
5  127.063196  
6  127.028601  
7  127.060275  
8  127.060651  
9  127.033061 

C:\Users\Ssoony\AppData\Local\Temp\ipykernel_22076\2293126012.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query,mydb)


In [241]:
import googlemaps


In [242]:
gmaps_key='....'
gmaps=googlemaps.Client(key=gmaps_key)

In [243]:
driver=webdriver.Chrome('../driver/chromedriver.exe')
driver.get('https://www.ediya.com/contents/find_store.html')
time.sleep(2)

In [244]:
query='SELECT DISTINCT gu_name FROM COFFEE_STORE WHERE brand = 1'
df = pd.read_sql(query,mydb)
gu_list=df.gu_name.to_list()

C:\Users\Ssoony\AppData\Local\Temp\ipykernel_22076\2674224659.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query,mydb)


In [245]:
driver.find_element_by_xpath('//*[@id="contentWrap"]/div[3]/div/div[1]/ul/li[2]/a').click()

In [246]:
for gu in tqdm_notebook(gu_list):
    driver.find_element_by_id('keyword').clear()
    driver.find_element_by_id('keyword').send_keys(f'서울 {gu}')
    driver.find_element_by_id('keyword').submit()
    time.sleep(2)

    ediyaCode=driver.page_source
    bs=BeautifulSoup(ediyaCode,'lxml')
    ediya_gu=bs.find('ul',id='placesList')
    ediyaRawData=ediya_gu.find_all('dl')

    for num2 in ediyaRawData:
        store_name=num2.find('dt').text
        store_address=num2.find('dd').text
        store_guname=store_address.split(' ')[1]
        try :
            tmp=gmaps.geocode(store_address, language='ko')
            store_lat=tmp[0].get('geometry')['location']['lat']
            store_lng=tmp[0].get('geometry')['location']['lng']
        except:
            last_adress=store_address[:store_address.index('(')]
            tmp=gmaps.geocode(last_adress, language='ko')
            store_lat=tmp[0].get('geometry')['location']['lat']
            store_lng=tmp[0].get('geometry')['location']['lng'] 
        cur.execute(f'INSERT INTO COFFEE_STORE(brand,name,gu_name,address,lat,lng) VALUES(2,\"{store_name}\",\"{store_guname}\",\"{store_address}\",{store_lat},{store_lng})')
        mydb.commit()    
    time.sleep(1)

C:\Users\Ssoony\AppData\Local\Temp\ipykernel_22076\148929880.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for gu in tqdm_notebook(gu_list):


  0%|          | 0/25 [00:00<?, ?it/s]

In [247]:
driver.close()

In [251]:
query='select COUNT(*) from COFFEE_STORE where brand=2'
df = pd.read_sql(query,mydb)
print(df)

   COUNT(*)
0       711


C:\Users\Ssoony\AppData\Local\Temp\ipykernel_22076\3429644352.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query,mydb)


In [278]:
query='select * from COFFEE_STORE where brand=2 limit 10'
df = pd.read_sql(query,mydb)
print(df)

    id  brand         name gu_name                          address  \
0  599      2      강남YMCA점     강남구                       서울 강남구 논현동   
1  600      2  강남구청역아이티웨딩점     강남구      서울 강남구 학동로 338 (논현동, 강남파라곤)   
2  601      2      강남논현학동점     강남구          서울 강남구 논현로131길 28 (논현동)   
3  602      2        강남대치점     강남구       서울 강남구 역삼로 415 (대치동, 성진빌딩)   
4  603      2        강남도산점     강남구          서울 강남구 도산대로37길 20 (신사동)   
5  604      2        강남세곡점     강남구  서울 강남구 헌릉로569길 9 (세곡동, 강남지웰파인즈)   
6  605      2        강남율현점     강남구    서울 강남구 밤고개로21길 8 (율현동, 세곡프라자)   
7  606      2        강남자곡점     강남구   서울 강남구 자곡로 180 (자곡동, 강남유탑유블레스)   
8  607      2        개포동역점     강남구     서울 강남구 개포로82길 11 (개포동, 삼우빌딩)   
9  608      2        건설회관점     강남구             서울 강남구 언주로 711 (논현동)   

         lat         lng  
0  37.513530  127.031534  
1  37.516466  127.040419  
2  37.514227  127.028666  
3  37.501251  127.052382  
4  37.522241  127.031481  
5  37.466973  127.100526  
6  37.469659  127.113020  
7 

C:\Users\Ssoony\AppData\Local\Temp\ipykernel_22076\3808716761.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query,mydb)


 - 스타벅스 매장 주요 분포 지역 (매장수가 많은 상위 5개 구이름, 매장 개수 출력)

In [279]:
query='select gu_name, count(*) as Total_Store from COFFEE_STORE where brand=1 group by gu_name order by count(*) desc limit 5'
df = pd.read_sql(query,mydb)
print(df)

  gu_name  Total_Store
0     강남구           88
1      중구           55
2     서초구           48
3    영등포구           41
4     종로구           40


C:\Users\Ssoony\AppData\Local\Temp\ipykernel_22076\1425266769.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query,mydb)


- 이디야 매장 주요 분포 지역 (매장수가 많은 상위 5개 구이름, 매장 개수 출력)

In [280]:
query='select gu_name, count(*) as Total_Store from COFFEE_STORE where brand=2 group by gu_name order by count(*) desc limit 5'
df = pd.read_sql(query,mydb)
print(df)

  gu_name  Total_Store
0     강남구           45
1     강서구           42
2    영등포구           41
3     송파구           39
4     마포구           36


C:\Users\Ssoony\AppData\Local\Temp\ipykernel_22076\1015218344.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query,mydb)


- 구별 브랜드 각각의 매장 개수 조회 (구이름, 브랜드이름, 매장 개수 출력)

In [281]:
query='select s.gu_name, b.name, count(*) as total_store from COFFEE_BRAND b join COFFEE_STORE s on b.id=s.brand group by s.gu_name, b.name order by s.gu_name, s.brand'
df = pd.read_sql(query,mydb)
print(df)

   gu_name       name  total_store
0      강남구  STARBUCKS           88
1      강남구      EDIYA           45
2      강동구  STARBUCKS           17
3      강동구      EDIYA           26
4      강북구  STARBUCKS            6
5      강북구      EDIYA           20
6      강서구  STARBUCKS           25
7      강서구      EDIYA           42
8      관악구  STARBUCKS           12
9      관악구      EDIYA           28
10     광진구  STARBUCKS           19
11     광진구      EDIYA           24
12     구로구  STARBUCKS           13
13     구로구      EDIYA           29
14     금천구  STARBUCKS           13
15     금천구      EDIYA           23
16     노원구  STARBUCKS           14
17     노원구      EDIYA           30
18     도봉구  STARBUCKS            4
19     도봉구      EDIYA           24
20    동대문구  STARBUCKS           10
21    동대문구      EDIYA           25
22     동작구  STARBUCKS           11
23     동작구      EDIYA           22
24     마포구  STARBUCKS           35
25     마포구      EDIYA           36
26    서대문구  STARBUCKS           22
27    서대문구      EDIY

C:\Users\Ssoony\AppData\Local\Temp\ipykernel_22076\381225149.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query,mydb)


 - 구별 브랜드 각각의 매장 개수 조회 (구이름, 스타벅스 매장 개수, 이디야 매장 개수 출력)

In [282]:
query='select gu_name, count(case when brand=1 then 1 end) as STARBUCKS, count(case when brand=2 then 1 end) as EDIYA from COFFEE_STORE group by gu_name'
df = pd.read_sql(query,mydb)
print(df)

   gu_name  STARBUCKS  EDIYA
0      강남구         88     45
1      강북구          6     20
2      강서구         25     42
3      관악구         12     28
4      광진구         19     24
5      금천구         13     23
6      노원구         14     30
7      도봉구          4     24
8      동작구         11     22
9      마포구         35     36
10    서대문구         22     21
11     서초구         48     29
12     성북구         15     31
13     송파구         34     39
14     양천구         17     26
15    영등포구         41     41
16     은평구         11     25
17     종로구         40     31
18      중구         55     30
19     강동구         17     26
20     구로구         13     29
21    동대문구         10     25
22     성동구         15     22
23     용산구         25     14
24     중랑구          8     28


C:\Users\Ssoony\AppData\Local\Temp\ipykernel_22076\1081149903.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query,mydb)


In [294]:
cur.execute('INSERT INTO STARBUCKS SELECT * FROM COFFEE_STORE WHERE brand=1')
cur.execute('INSERT INTO EDIYA SELECT * FROM COFFEE_STORE WHERE brand=2')

In [296]:
mydb.commit()

In [295]:
query=("select s.id as s_id, case s.brand when 1 then 'STARBUCKS' else 'EDIYA' end as s_brand, s.name as s_name, s.gu_name as s_gu, s.address as s_address, s.lat as s_lat, s.lng as s_lng, e.id as d_id, case e.brand when 1 then 'STARBUCKS' else 'EDIYA' end as e_brand, e.name as e_name, e.gu_name as e_gu, e.address as e_address, e.lat as e_lat, e.lng as e_lng from STARBUCKS s, EDIYA e order by s.id, e.id")
df = pd.read_sql(query,mydb)
print(df)

C:\Users\Ssoony\AppData\Local\Temp\ipykernel_22076\2792974278.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query,mydb)


        s_id    s_brand     s_name s_gu                s_address      s_lat  \
0          1  STARBUCKS  역삼아레나빌딩    강남구  서울특별시 강남구 언주로 425 (역삼동)  37.501087   
1          1  STARBUCKS  역삼아레나빌딩    강남구  서울특별시 강남구 언주로 425 (역삼동)  37.501087   
2          1  STARBUCKS  역삼아레나빌딩    강남구  서울특별시 강남구 언주로 425 (역삼동)  37.501087   
3          1  STARBUCKS  역삼아레나빌딩    강남구  서울특별시 강남구 언주로 425 (역삼동)  37.501087   
4          1  STARBUCKS  역삼아레나빌딩    강남구  서울특별시 강남구 언주로 425 (역삼동)  37.501087   
...      ...        ...        ...  ...                      ...        ...   
425173   598  STARBUCKS      중화역    중랑구       서울특별시 중랑구 봉화산로 35   37.601709   
425174   598  STARBUCKS      중화역    중랑구       서울특별시 중랑구 봉화산로 35   37.601709   
425175   598  STARBUCKS      중화역    중랑구       서울특별시 중랑구 봉화산로 35   37.601709   
425176   598  STARBUCKS      중화역    중랑구       서울특별시 중랑구 봉화산로 35   37.601709   
425177   598  STARBUCKS      중화역    중랑구       서울특별시 중랑구 봉화산로 35   37.601709   

             s_lng  d_id e_brand       e_name e_gu 

In [298]:
df.to_csv('C:/Users/Ssoony/Documents/ds_study/data/07.COFFEE_BRAND_DATA.csv', sep=',', encoding='euc-kr')